In [1]:
# https://medium.com/@akriti.upadhyay/efficient-information-retrieval-with-rag-workflow-afdfc2619171
# https://medium.com/@akriti.upadhyay/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7

In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [ ]:
# Specify the dataset name and the column containing the content
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context" # or any other column you're interested in 

# Create a loader instance 
loader = HuggingFaceDatasetLoader(dataset_name,page_content_column)
# Load the data
data = loader.load()

# Display the first entries
data[:2]

In [ ]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)

# 'data' holds the text you want to split, split the text into document using the text splitter.
docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}
# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

In [ ]:
db = FAISS.from_documents(docs,embeddings)

In [ ]:
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

In [ ]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert"
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinydert" model
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

In [ ]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model 
tokenizer = AutoTokenizer.from_pretrained(model_name,padding=True,truncation=True,max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature":0.7,"max_length":512}
)

In [ ]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or document from the database
retriever = db.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("What is Cheeseemaking?")
print(docs[0].page_content)

In [ ]:
print(docs[0].page_content)

In [ ]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant split/documents
retriever = db.as_retriever(search_kwargs={"k":4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine", the retriever we createed, and option to not return source documents
qa = RetrievalQA.from_chain_type(llm=llm,chain_type="refine",retriever=retriever,return_source_documents=False)

In [ ]:
qa

In [ ]:
question = "Who is Thomas Jefferson?"

In [ ]:
def err_remove(er):
    lin = "------------"
    er = str(er)
    start_index = er.find(lin) + len(lin)
    end_index = er.rfind(lin)
    Answer = er[start_index:end_index].strip()
    return Answer

In [ ]:
import sys

In [ ]:
try:
    result = qa.invoke({"query": question})
    result["result"]
except:
    _,error,_ = sys.exc_info()
    answer = err_remove(error)

In [ ]:
answer